# map2loop:  Process plutons

For each instruve but not sill polygon, find older neighbours and store decimated contact points. Also store dipping contact orientations (user defined, just because) with three possible sub-surface configurations:

<b>saucers: \\_____/ <br>
pancakes: /_____\\   
domes: /‾\\</b>

Saves out orientations and contact points, as well as updated group level stratigraphic column.<br>
Doesn't account for faults yet...

In [1]:
import matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import numpy as np
from matplotlib.pyplot import imshow
import rasterio
from rasterio.transform import from_origin
from rasterio import features
%matplotlib inline
import sys, os
from map2loop import m2l_utils
import geopandas as gpd
from math import acos, sqrt, cos, sin, degrees, radians, fabs, atan2
from shapely.geometry import shape, Polygon, LineString, Point
## import geopandas as gpd
#os.chdir('..')

print(os.getcwd())


C:\Users\00073294\Dropbox\1_Jupyter_notebooks\map2loop\notebooks


In [2]:
%run -i "../notebooks/m2l_config.py"
bbox=(minx,miny,maxx,maxy)

pluton_dip=str(pluton_dip)

dist_buffer=10



default parameters loaded from m2l_config.py


In [3]:


geology = gpd.read_file('../test_data3/data/hams2_geol.shp',bbox=bbox)
print(geology.crs)

geol_explode=m2l_utils.explode(geology)
print(geology.shape, geol_explode.shape)
roi = gpd.read_file('../test_data3/data/hamms_roi.shp')

geol_clip=m2l_utils.clip_shp(geol_explode,roi)


dtm_reproj_file='../test_data3/dtm/hams3_dtm_rp.tif'

dtm = rasterio.open(dtm_reproj_file)

f=open('../test_data3/tmp/hams3_groups.csv',"r")
groups =f.readlines()
f.close

ngroups=groups[0].split(" ")
ngroups=int(ngroups[1])

orig_ngroups=ngroups




{'init': 'epsg:28350'}
(186, 63) (186, 63)


In [4]:
gp_ages=np.zeros((1000,3))
gp_names=np.zeros((1000),dtype='U25')

for i in range (0,ngroups):
    gp_ages[i,0]=-1e6 # group max_age
    gp_ages[i,1]=1e6 # group min_age
    gp_ages[i,2]=i # group index
    gp_names[i]=groups[i+1].replace("\n","")
    print(i,gp_names[i])

    

0 Wyloo_Group
1 Shingle_Creek_Group
2 Hamersley_Group
3 Turee_Creek_Group
4 Fortescue_Group
5 A_s_PMI
6 A_mgn_PMI
7 A_mgn_PRK
8 A_s_PRK
9 A_b_PRK


In [7]:
allc=open(output_path+'all_ign_contacts.csv',"w")
allc.write('GROUP_,id,x,y,z,code\n')
ac=open(output_path+'ign_contacts.csv',"w")
ac.write("X,Y,Z,formation\n")
ao=open(output_path+'ign_orientations_'+pluton_form+'.csv',"w")
ao.write("X,Y,Z,azimuth,dip,polarity,formation\n")

j=0
allpts=0
ls_dict={}
ls_dict_decimate={}
id=0
for ageol in geol_clip.iterrows(): # central polygon
    ades=str(ageol[1][dscode])
    arck=str(ageol[1][r1code])
    if(str(ageol[1][gcode])=='None'):
        agroup=str(ageol[1][ccode])
    else:
        agroup=str(ageol[1][gcode])
    
    for i in range(0,ngroups):
        if (gp_names[i]==agroup):
            if(int(ageol[1][maxcode]) > gp_ages[i][0]  ):
                gp_ages[i][0] = ageol[1][maxcode]
                #print("max",agroup,gp_ages[i][0])
            if(int(ageol[1][mincode]) < gp_ages[i][1]  ):
                gp_ages[i][1] = ageol[1][mincode]
                #print("min",agroup,gp_ages[i][1])
    if('intrusive' in arck and 'sill' not in ades):
        newgp=str(ageol[1][ccode])+'_'+str(ageol[1][ocode])
        #agp=str(ageol[1][gcode])
        #print(newgp)
        if(str(ageol[1][gcode])=='None'):
            agp=str(ageol[1][ccode])
        else:
            agp=str(ageol[1][gcode])

        if(not newgp  in gp_names):
            #print("MMMMM",ngroups,newgp)
            gp_names[ngroups]=newgp
            gp_ages[ngroups][0]=ageol[1][maxcode]
            gp_ages[ngroups][1]=ageol[1][mincode]
            gp_ages[ngroups][2]=ngroups
            ngroups=ngroups+1
        #else:
            #print("-----",ngroups,newgp)
            
        neighbours=[]
        j+=1
        central_age=ageol[1][mincode]    #absolute age of central polygon
        central_poly=ageol[1].geometry
        for bgeol in geol_clip.iterrows(): #potential neighbouring polygons  
            if(ageol[1].geometry!=bgeol[1].geometry): #do not compare with self
                if (ageol[1].geometry.intersects(bgeol[1].geometry)): # is a neighbour
                    neighbours.append([(bgeol[1][ccode],bgeol[1][mincode],bgeol[1][r1code],bgeol[1][dscode],bgeol[1].geometry)])  
        #display(neighbours)
        if(len(neighbours) >0):
            for i in range (0,len(neighbours)):
                if(('intrusive' in neighbours[i][0][2] and 'sill' not in ades) 
                   #or ('intrusive' not in neighbours[i][0][2]) and neighbours[i][0][1] > central_age ): # neighbour is older than central
                   or ('intrusive' not in neighbours[i][0][2]) and neighbours[i][0][1]  ): # neighbour is older than central
                    #print(ageol[1][ccode],neighbours[i][0][0])
                    older_polygon=neighbours[i][0][4]
                    if(not central_poly.is_valid ):
                        central_poly = central_poly.buffer(0)
                    if(not older_polygon.is_valid):
                        older_polygon = older_polygon.buffer(0)
                    LineStringC = central_poly.intersection(older_polygon)
                    if(LineStringC.wkt.split(" ")[0]=='GEOMETRYCOLLECTION' or 
                       LineStringC.wkt.split(" ")[0]=='MULTIPOLYGON' or
                       LineStringC.wkt.split(" ")[0]=='POLYGON'): #ignore polygon intersections for now, worry about them later!
                        #print("debug:GC,MP,P")
                        continue

                    elif(LineStringC.wkt.split(" ")[0]=='MULTILINESTRING'):
                        k=0
                        #print("lenlenlen",len(LineStringC))

                        #display(LineStringC)
                        ls_dict[id] = {"id": id,"CODE":newgp,"GROUP_":newgp, "geometry": LineStringC}
                        id=id+1
                        for lineC in LineStringC: #process all linestrings
                            #if(contact_decimate!=0): #decimate to reduce number of points
                            if(m2l_utils.mod_safe(k,contact_decimate)==0 or k==int((len(LineStringC)-1)/2) or k==len(LineStringC)-1): #decimate to reduce number of points, but also take second and third point of a series to keep gempy happy
                                locations=[(lineC.coords[0][0],lineC.coords[0][1])] #doesn't like point right on edge?
                                #print(k,type(lineC))
                                if(lineC.coords[0][0] > dtm.bounds[0] and lineC.coords[0][0] < dtm.bounds[2] and  
                                   lineC.coords[0][1] > dtm.bounds[1] and lineC.coords[0][1] < dtm.bounds[3]):       
                                        height=m2l_utils.value_from_raster(dtm,locations)
                                        ostr=str(lineC.coords[0][0])+","+str(lineC.coords[0][1])+","+height+","+newgp+"\n"
                                        ac.write(ostr)
                                        allc.write(agp+","+str(ageol[1][ocode])+","+ostr)
                                        ls_dict_decimate[allpts] = {"id": allpts,"CODE":newgp,"GROUP_":newgp, "geometry": Point(lineC.coords[0][0],lineC.coords[0][1])}
                                        allpts+=1 
                                else:
                                    continue
                                    #print("debug:edge points")
                            else:
                                if(lineC.coords[0][0] > dtm.bounds[0] and lineC.coords[0][0] < dtm.bounds[2] and  
                                        lineC.coords[0][1] > dtm.bounds[1] and lineC.coords[0][1] < dtm.bounds[3]):       
                                    height=m2l_utils.value_from_raster(dtm,locations)
                                    ostr=str(lineC.coords[0][0])+","+str(lineC.coords[0][1])+","+height+","+newgp+"\n"
                                    #ls_dict_decimate[allpts] = {"id": id,"CODE":ageol[1]['CODE'],"GROUP_":ageol[1]['GROUP_'], "geometry": Point(lineC.coords[0][0],lineC.coords[0][1])}
                                    allc.write(agp+","+str(ageol[1][ocode])+","+ostr)
                                    allpts+=1
                            
                            #print(m2l_utils.mod_safe(k,contact_decimate))
                            if(m2l_utils.mod_safe(k,contact_decimate)==0 or k==int((len(LineStringC)-1)/2) or k==len(LineStringC)-1): #decimate to reduce number of points, but also take second and third point of a series to keep gempy happy
                                dlsx=lineC.coords[0][0]-lineC.coords[1][0]
                                dlsy=lineC.coords[0][1]-lineC.coords[1][1]
                                lsx=dlsx/sqrt((dlsx*dlsx)+(dlsy*dlsy))
                                lsy=dlsy/sqrt((dlsx*dlsx)+(dlsy*dlsy))                                        

                                locations=[(lineC.coords[0][0],lineC.coords[0][1])]
                                height= m2l_utils.value_from_raster(dtm,locations)
                                azimuth=(180+degrees(atan2(lsy,-lsx)))%360 #normal to line segment
                                testpx=lineC.coords[0][0]+lsy # pt just a bit in/out from line
                                testpy=lineC.coords[0][0]+lsx

                                for cgeol in geol_clip.iterrows(): # check on direction to dip
                                    if LineString(central_poly.exterior.coords).contains(Point(testpx, testpy)):
                                        azimuth=(azimuth-180)%360
                                        break
                                if(pluton_form=='saucers'):
                                    ostr=str(lineC.coords[0][0])+","+str(lineC.coords[0][1])+","+str(height)+","+str(azimuth)+","+pluton_dip+",1,"+newgp+"\n"
                                elif(pluton_form=='domes'):
                                    azimuth=(azimuth-180)%360
                                    ostr=str(lineC.coords[0][0])+","+str(lineC.coords[0][1])+","+str(height)+","+str(azimuth)+","+pluton_dip+",0,"+newgp+"\n"
                                elif(pluton_form=='dontknow'):
                                    ostr=str(lineC.coords[0][0])+","+str(lineC.coords[0][1])+","+str(height)+","+str(azimuth)+","+pluton_dip+",0,"+newgp+"\n"
                                else: #pluton_form == pancakes
                                    azimuth=(azimuth-180)%360
                                    ostr=str(lineC.coords[0][0])+","+str(lineC.coords[0][1])+","+str(height)+","+str(azimuth)+","+pluton_dip+",1,"+newgp+"\n"
                                    
                                ao.write(ostr)

                            k+=1
                    elif(LineStringC.wkt.split(" ")[0]=='LINESTRING'): # apparently this is not needed
                        #print("debug:LINESTRING")
                        k=0
                        for pt in LineStringC.coords: #process one linestring
                            #if(i%contact_decimate==0): #decimate to reduce number of points
                            #print("ls",pt)
                            k+=1
                    elif(LineStringC.wkt.split(" ")[0]=='POINT'): # apparently this is not needed
                        #print("debug:POINT")
                        #print("pt",LineStringC.coords)
                        k+=1
                    else:
                        #print(LineStringC.wkt.split(" ")[0]) # apparently this is not needed
                        k+=1
ac.close()
ao.close()
allc.close()

#print(ngroups)
#for i in range (0,ngroups):
#    print(i,gp_names[i])

#display(gp_ages[:ngroups])
#display(gp_names[:ngroups])

#ga=gp_ages[:ngroups]
#print("XXXXXXXXXXXXX",ga)
#f=ga[:,0].argsort()
#display(f)
  
an=open('../test_data3/tmp/hams3_groups2.csv',"w")
an.write('1 '+str(ngroups)+'\n')
for i in range (orig_ngroups,ngroups):
    print(i,gp_names[i])
    an.write(gp_names[i]+'\n')
for i in range (0,orig_ngroups):
    print(i,gp_names[i])
    an.write(gp_names[i]+'\n')
an.close()

10 A-mgn-PMI_520
11 A-mgn-PRK_32
0 Wyloo_Group
1 Shingle_Creek_Group
2 Hamersley_Group
3 Turee_Creek_Group
4 Fortescue_Group
5 A_s_PMI
6 A_mgn_PMI
7 A_mgn_PRK
8 A_s_PRK
9 A_b_PRK
